# Imports


In [ ]:
import os
import subprocess
from pathlib import Path

# Params (user should edit/check this)


In [ ]:
model = "htdemucs"  # model options can be found at https://github.com/facebookresearch/demucs (main choices are 'htdemucs', 'htdemucs_ft', 'mdx_extra')
extensions = ["mp3", "wav", "ogg", "flac"]  # we will look for all those file types.
two_stems = (
    None  # only separate one stems from the rest, for instance vocals from the rest.
)

# Options for the output audio.
mp3 = False
mp3_rate = 320
shifts = 10  # performs multiple predictions with random shifts (a.k.a the shift trick) of the input and average them. This makes prediction SHIFTS times slower.
overlap = 0.25  # controls the amount of overlap between prediction windows.
float32 = False  # output as float 32 wavs, unsused if 'mp3' is True.
int24 = False  # output as int24 wavs, unused if 'mp3' is True.
# You cannot set both `float32 = True` and `int24 = True`

in_path = "input_audio"
out_path = "output_audio"

# Setup


In [ ]:
def find_files(in_path):
    out = []
    for file in Path(in_path).iterdir():
        if file.suffix.lower().lstrip(".") in extensions:
            out.append(file)
    return out


def separate(inp=None, outp=None):
    inp = inp or in_path
    outp = outp or out_path
    cmd = [
        "python3",
        "-m",
        "demucs.separate",
        "-o",
        str(outp),
        "-n",
        model,
        "--shifts",
        str(shifts),
        "--overlap",
        str(overlap),
    ]

    if mp3:
        cmd += ["--mp3", f"--mp3-bitrate={mp3_rate}"]
    if float32:
        cmd += ["--float32"]
    if int24:
        cmd += ["--int24"]
    if two_stems is not None:
        cmd += [f"--two-stems={two_stems}"]

    files = [str(f) for f in find_files(inp)]
    if not files:
        print(f"No valid audio files in {in_path}")
        return

    print("Going to separate the files:")
    print("\n".join(files))
    print("With command: ", " ".join(cmd))

    try:
        # Use binary mode and handle encoding manually
        process = subprocess.Popen(
            cmd + files,
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT,  # Merge stderr into stdout
            bufsize=1,
            env={
                **os.environ,
                "PYTHONIOENCODING": "utf-8",
            },  # Ensure Python subprocess uses UTF-8
        )

        # Read and decode output
        while True:
            output = process.stdout.readline()
            if not output and process.poll() is not None:
                break
            if output:
                try:
                    # Decode with utf-8, fallback to cp1252 if needed, ignore problematic characters
                    line = output.decode("utf-8", errors="replace")
                    print(line.strip())
                except Exception as e:
                    print(f"Warning: Could not decode output line: {e}")

        returncode = process.poll()
        if returncode != 0:
            print(f"Command failed with return code {returncode}")

    except Exception as e:
        print(f"An error occurred: {e}")
        if "process" in locals():
            process.kill()
        raise

# Inference


In [ ]:
separate()